## Week list

In [1]:
import os
import random
import yaml
import math
from collections import defaultdict
from datetime import date

import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np

from src.Settings import RECIPE_DIR, WEEKLY_LIST_DIR
from src.Recipe import SCALING_FACTORS, list_recipes, get_ingredient_unit, format_ingredient_name, update_selection, Recipe
from src.RecipeSelection import make_selection
from src.WeeklyList import WeeklyList, WeeklyListParser

recipe_list = list_recipes()
recipe_names = list(recipe_list.keys())

today = date.today()
recipe_names_with_selection = []
selection_array = []
for name, recipe in recipe_list.items():
    recipe_obj =  Recipe.load(recipe["filename"])
    score = recipe_obj.compute_seasonal_score()
    if recipe.get("selection"):
        recipe_names_with_selection.append(f"{(today - recipe.get('selection')).days} - {name} -   {score}")
        selection_array.append([(today - recipe.get('selection')).days, score])
    else:
        recipe_names_with_selection.append(f"Never - {name} -   {score}")
        selection_array.append([365, score])
        
recipe_names_with_selection = sorted(recipe_names_with_selection)
make_selection(recipe_names, selection_array)
        

this_week_list = widgets.VBox(
    [
        widgets.BoundedIntText(value="7", min=0, max=10, description="Number of recipes:"),
        widgets.HBox([widgets.Dropdown(options=recipe_names_with_selection,
                                description='Select',
                                disabled=False), widgets.Button(disabled=False, button_style='', tooltip='Add', icon='plus')]),
        widgets.Label(value="This week's list: "),
        widgets.VBox([]),
        widgets.Button(
            disabled=False,
            button_style='',
            tooltip='Generate',
            icon='play',
        ),
        widgets.Button(
            disabled=False,
            button_style='',
            tooltip='Shopping list',
            icon='store',
        ),
    ]
)

def add_button(b=None):
    print(this_week_list.children[1].children[0].value)
    add_recipe(this_week_list.children[1].children[0].value.split(" - ")[1])

def change_button(b=None):
    selection = random.randint(0, len(recipe_names)-1)
    this_week_list.children[3].children[int(b.tooltip)].children[-1].value = f"{recipe_names[selection]}"

def remove_button(b=None):
    print(b.tooltip)
    this_week_list.children[3].children[int(b.tooltip)].children=()

def add_recipe(name):
    existing_names  = [child.children[-1].value for child in this_week_list.children[3].children]
    if name in existing_names:
        print("Warning: recipe already included. Skipping.")
        return False
    i = len(this_week_list.children[3].children)
    this_week_list.children[3].children += (widgets.HBox(
        [
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip=str(i),
                icon='pen',
            ),
            widgets.Button(
                disabled=False,
                button_style='',
                tooltip=str(i),
                icon='minus',
            ),
            widgets.Label(value=f"{name}"),
        ]),
    )
    this_week_list.children[3].children[-1].children[0].on_click(change_button)
    this_week_list.children[3].children[-1].children[1].on_click(remove_button)
    return True
    

def generate_recipes(b):
    if this_week_list.children[3].children:
        this_week_list.children[3].children = ()
    for i in range(this_week_list.children[0].value):
        selection = random.randint(0, len(recipe_names)-1)
        while not add_recipe(recipe_names[selection]):
              selection = random.randint(0, len(recipe_names)-1)

def get_general_ingredient_name(name, ingredient_dict):
    for ingredient_name, ingredient_obj in ingredient_dict.items():
        possible_names = [format_ingredient_name(ingredient_name)]
        for ingredient_alt in ingredient_obj.get("alternatives", []):
            possible_names.append(format_ingredient_name(ingredient_alt))
        if format_ingredient_name(name) in possible_names:
            return ingredient_name
    
    raise Exception(f"Ingredient '{name}' not known. Please add it. {possible_names}")

def create_shopping_list(b=None):
    
    # retrieve existing ingredients
    ingredient_dict = None
    with open(os.path.join(os.getcwd(),"rsrc", "ingredients.yaml"), 'r') as ingf:
        ingredient_dict = yaml.load(ingf, Loader=yaml.FullLoader)
    
    # Process all recipes selected, extract ingredients and add then to the shopping list
    shopping_dict = {}
    weekly_list = []
    for recipe_widget in this_week_list.children[3].children:
        recipe = Recipe.load(recipe_list[recipe_widget.children[-1].value]["filename"], recipe_list[recipe_widget.children[-1].value]["id"])
        for recipe_ingredient in recipe.ingredients:
            name = get_general_ingredient_name(recipe_ingredient["name"], ingredient_dict)
            if recipe_ingredient.get("description"):
                name += " (" + recipe_ingredient.get("description") + ")"
            
            unit, value = get_ingredient_unit(recipe_ingredient)
            if unit is None or value is None:
                raise Exception(f"Ingredient {ingredient} with None value (from {recipe_widget.children[-1].value})")
            
            if name in shopping_dict.keys():
                if unit != shopping_dict[name]["unit"]:
                    raise Exception(f"Ingredients with different units. {name} ({unit}/{shopping_dict[name]['unit']})" )
                shopping_dict[name]["value"] += value
            else:
                shopping_dict[name] = {"unit": unit, "value": value}
        update_selection(recipe.name)
        weekly_list.append(recipe.name)
    
    # saving the shopping list and this week's meals
    shopping_list = []
    for name, content in shopping_dict.items():
        if content["unit"] == "x":
            shopping_list.append(f"{name} x {math.ceil(content['value'])}")
        else:
            unit = content['unit']
            value = content['value']
            for k, v in SCALING_FACTORS.items():
                if (v > 1.0 and value / v >= 1.0) or (v <= 1.0 and value / v >= 1.0):
                    unit = k + unit
                    value = value / v
                    break

            shopping_list.append(f"{value}{unit} {name}")
            
    ingredient_list = WeeklyList("ShoppingList")
    ingredient_list.save(shopping_list)
    
    meal_list = WeeklyList("Meals")
    meal_list.update(weekly_list)
                
    print("Shopping list created successfully!")
    
this_week_list.children[1].children[-1].on_click(add_button)        
this_week_list.children[-2].on_click(generate_recipes)
this_week_list.children[-1].on_click(create_shopping_list)
display(this_week_list)

weekly_list_parser = WeeklyListParser()
weekly_list_parser.parse(os.path.join(WEEKLY_LIST_DIR, "Meals") + ".sqd")
week_recipes = weekly_list_parser.get_todo_list()["todo"]
for recipe in week_recipes:
    add_recipe(recipe)

                             days  score
Fish cake                   365.0   0.75
Tarte brocolis camembert    365.0   0.75
Gnocchi Roquefort           365.0   0.00
Petits pois au chorizo      365.0   0.00
Poulet miel balsamique      365.0   0.00
Choufleur Curry Pommes      365.0  -0.25
Tomate Chorizo              365.0  -1.00
Thai green/red curry         13.0   0.00
croques monsieur             13.0   0.00
Broccoli Quinoa              12.0   0.75
Butternut chèvre             12.0   0.75
Smoked Salmon Spaghettis     12.0   0.75
Tarte aux poireaux           12.0   0.75
halloumi kebab               12.0   0.75
Brandade de Morue            12.0   0.00
Cake chèvre épinards         12.0   0.00
Fajitas                      12.0   0.00
Japanese Curry               12.0   0.00
Pasta Bolognese              12.0   0.00
Stuffed Peppers              12.0   0.00
Tagliatelle alla carbonara   12.0   0.00
Tortellini                   12.0   0.00
Wok Salmon Spinach           12.0   0.00
test            

In [2]:
display(widgets.HTML(
    value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/show_recipes.ipynb'><b>Show recipes</b></a>"
))
display(widgets.HTML(
    value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/create_recipe.ipynb'><b>Create or edit recipe</b></a>"
))

HTML(value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/show_recipes.ipynb'><b>Show recipes</…

HTML(value="<a href='http://192.168.1.133:8080/voila/render/git/PyRecipe/create_recipe.ipynb'><b>Create or edi…

In [3]:
import os

if "ANDROID_ARGUMENT" in os.environ:
    print("Android system")
else:
    print("Linux system")

Linux system
